In [ ]:
!pip install mlxtend

# Generated Groups

In [1]:
import pandas as pd
from mlxtend.frequent_patterns import fpgrowth
from mlxtend.preprocessing import TransactionEncoder
from sklearn.metrics.pairwise import cosine_similarity
from sentence_transformers import SentenceTransformer
import numpy as np
import joblib

C:\Users\Krlozz\anaconda3\lib\site-packages\sentence_transformers\cross_encoder\CrossEncoder.py:11: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange
C:\Users\Krlozz\anaconda3\lib\site-packages\transformers\utils\generic.py:441: FutureWarning: `torch.utils._pytree._register_pytree_node` is deprecated. Please use `torch.utils._pytree.register_pytree_node` instead.
  _torch_pytree._register_pytree_node(
C:\Users\Krlozz\anaconda3\lib\site-packages\transformers\utils\generic.py:309: FutureWarning: `torch.utils._pytree._register_pytree_node` is deprecated. Please use `torch.utils._pytree.register_pytree_node` instead.
  _torch_pytree._register_pytree_node(


In [2]:
articles_authors_path = "C:/Users/Krlozz/Documents/Tesis/data_JA/articles_authors.csv"
keywords_path = "C:/Users/Krlozz/Documents/Tesis/TesisFinal/ProcessedData/generated_keywords_articles.csv"
output_path = "C:/Users/Krlozz/Documents/Tesis/TesisFinal/ProcessedData/generated_groups.csv"
model_sentece_path = "C:/Users/Krlozz/Documents/Tesis/TesisFinal/Model/sentence_bert_model"

In [3]:
articles_authors_df = pd.read_csv(articles_authors_path, header=None, names=['article_id', 'author_id'])
keywords_df = pd.read_csv(keywords_path)

In [4]:
articles_grouped = articles_authors_df.groupby('article_id')['author_id'].apply(list).tolist()

In [5]:
te = TransactionEncoder()
te_ary = te.fit(articles_grouped).transform(articles_grouped)
df = pd.DataFrame(te_ary, columns=te.columns_)
frequent_itemsets = fpgrowth(df, min_support=3/len(df), use_colnames=True)

In [6]:
valid_groups = frequent_itemsets[frequent_itemsets['itemsets'].apply(lambda x: len(x) >= 3)].reset_index(drop=True)
valid_groups['id_group'] = ['group' + str(i+1) for i in range(len(valid_groups))]
valid_groups['authors'] = valid_groups['itemsets'].apply(lambda x: ', '.join(map(str, x)))

In [7]:
article_author_df = articles_authors_df.groupby('article_id')['author_id'].apply(set).reset_index()

In [8]:
def find_articles_for_group(group, article_author_df):
    group_authors = set(group)
    matching_articles = article_author_df[article_author_df['author_id'].apply(lambda x: group_authors.issubset(x))]['article_id'].tolist()
    return matching_articles if len(matching_articles) >= 3 else []

In [9]:
valid_groups['id_articles_group'] = valid_groups['itemsets'].apply(lambda x: find_articles_for_group(x, article_author_df))

valid_groups = valid_groups[valid_groups['id_articles_group'].apply(len) >= 3].reset_index(drop=True)

In [10]:
valid_groups['id_articles_group']

0        [84885079328, 84946204179, 84962109938]
1         [0011957585, 50449094016, 70349170306]
2         [0011957585, 50449094016, 70349170306]
3         [0011957585, 50449094016, 70349170306]
4         [0011957585, 50449094016, 70349170306]
                          ...                   
17672    [85125384992, 85127342869, 85131248040]
17673    [85125384992, 85127342869, 85131248040]
17674    [85125384992, 85127342869, 85131248040]
17675    [85125384992, 85127342869, 85131248040]
17676    [85125384992, 85127342869, 85131248040]
Name: id_articles_group, Length: 17677, dtype: object

In [11]:
def find_all_articles(group, articles_authors_df, articles_in_group):
    articles_in_group_set = set(articles_in_group)
    all_articles = set()
    for author in group:
        author_articles = articles_authors_df[articles_authors_df['author_id'] == author]['article_id'].tolist()
        all_articles.update(author_articles)
    return list(all_articles - articles_in_group_set)

In [12]:
valid_groups['id_articles_all'] = valid_groups.apply(
    lambda row: find_all_articles(row['itemsets'], articles_authors_df, row['id_articles_group']), axis=1
)

In [13]:
valid_groups['id_articles_all']

0        [84864296662, 85093931360, 85055531776, 851317...
1        [85021017087, 0026699611, 0030764844, 85091217...
2        [0027731713, 0026653512, 0026699611, 003180785...
3        [85021017087, 0026699611, 0030764844, 85091217...
4        [85021017087, 0030764844, 85091217062, 0032509...
                               ...                        
17672    [85120535327, 85104328183, 85131537395, 851269...
17673    [85082691209, 85131537395, 85091110208, 850616...
17674    [85082691209, 85131537395, 85091110208, 850616...
17675    [85082691209, 85131537395, 85091110208, 850616...
17676    [85082691209, 85131537395, 85091110208, 850616...
Name: id_articles_all, Length: 17677, dtype: object

In [33]:
def extract_keywords(article_ids, keywords_df):
    keywords = set()
    for article_id in article_ids:
        
        article_id = str(article_id)
        matched_rows = keywords_df[keywords_df['id_article'] == article_id]
        if not matched_rows.empty:
            generated_keywords = matched_rows.iloc[0]['generated_keywords']
            if pd.notna(generated_keywords):
                keywords.update(generated_keywords.split(', '))
    return list(keywords)

In [34]:
valid_groups['keywords_group'] = valid_groups['id_articles_group'].apply(
    lambda articles: extract_keywords(articles, keywords_df)
)

In [35]:
valid_groups['keywords_group']

0        [ovt instructing new, acoustic flow monitors, ...
1        [countries differential access, disappearance ...
2        [countries differential access, disappearance ...
3        [countries differential access, disappearance ...
4        [countries differential access, disappearance ...
                               ...                        
17672    [mitochondrial transfer transplant, work preve...
17673    [mitochondrial transfer transplant, work preve...
17674    [mitochondrial transfer transplant, work preve...
17675    [mitochondrial transfer transplant, work preve...
17676    [mitochondrial transfer transplant, work preve...
Name: keywords_group, Length: 17677, dtype: object

In [36]:
valid_groups['keywords_all'] = valid_groups['id_articles_all'].apply(
    lambda articles: extract_keywords(articles, keywords_df)
)

In [37]:
valid_groups['keywords_all']

0        [teleseismic travel time, reduced computationa...
1        [location parasite extraparenchymal, marginali...
2        [review article suggest, resistant epilepsy su...
3        [location parasite extraparenchymal, marginali...
4        [location parasite extraparenchymal, marginali...
                               ...                        
17672    [hcps exist related, identification diversity ...
17673    [patch treat gastroschisis, highlight need sta...
17674    [patch treat gastroschisis, highlight need sta...
17675    [patch treat gastroschisis, highlight need sta...
17676    [patch treat gastroschisis, highlight need sta...
Name: keywords_all, Length: 17677, dtype: object

In [38]:
model_sentece = joblib.load(model_sentece_path)

In [39]:
def calculate_similarity(keywords_group, keywords_all):
    if not keywords_group or not keywords_all:
        return np.nan
    embeddings_group = model_sentece.encode(keywords_group)
    embeddings_all = model_sentece.encode(keywords_all)
    return cosine_similarity(
        np.mean(embeddings_group, axis=0).reshape(1, -1),
        np.mean(embeddings_all, axis=0).reshape(1, -1)
    )[0][0]

In [40]:
valid_groups['similarity_cosine'] = valid_groups.apply(
    lambda row: calculate_similarity(row['keywords_group'], row['keywords_all']), axis=1
)
valid_groups['index_novelty'] = 1 - valid_groups['similarity_cosine']

In [41]:
valid_groups['num_articles_group'] = valid_groups['id_articles_group'].apply(len)
valid_groups['num_articles_all'] = valid_groups['id_articles_all'].apply(len)
valid_groups['num_keywords_group'] = valid_groups['keywords_group'].apply(len)
valid_groups['num_keywords_all'] = valid_groups['keywords_all'].apply(len)

In [42]:
columns_to_save = [
    'id_group', 'authors', 'id_articles_group', 'num_articles_group',
    'id_articles_all', 'num_articles_all', 'keywords_group', 'num_keywords_group',
    'keywords_all', 'num_keywords_all', 'similarity_cosine', 'index_novelty'
]

In [43]:
valid_groups

,support,itemsets,id_group,authors,id_articles_group,id_articles_all,keywords_group,keywords_all,similarity_cosine,index_novelty,num_articles_group,num_articles_all,num_keywords_group,num_keywords_all
0,0.000075,"(7403483234, 57200218104, 6603155100)",group1,"7403483234, 57200218104, 6603155100","[84885079328, 84946204179, 84962109938]","[84864296662, 85093931360, 85055531776, 851317...","[ovt instructing new, acoustic flow monitors, ...","[teleseismic travel time, reduced computationa...",0.835043,0.164957,3,170,15,744
1,0.000075,"(24734179100, 7003513941, 7005237183)",group2,"24734179100, 7003513941, 7005237183","[0011957585, 50449094016, 70349170306]","[85021017087, 0026699611, 0030764844, 85091217...","[countries differential access, disappearance ...","[location parasite extraparenchymal, marginali...",0.771965,0.228035,3,68,10,187
2,0.000075,"(24734179100, 7005237183, 7102187693)",group3,"24734179100, 7005237183, 7102187693","[0011957585, 50449094016, 70349170306]","[0027731713, 0026653512, 0026699611, 003180785...","[countries differential access, disappearance ...","[review article suggest, resistant epilepsy su...",0.582562,0.417438,3,13,10,10
3,0.000075,"(7003513941, 7005237183, 7102187693)",group4,"7003513941, 7005237183, 7102187693","[0011957585, 50449094016, 70349170306]","[85021017087, 0026699611, 0030764844, 85091217...","[countries differential access, disappearance ...","[location parasite extraparenchymal, marginali...",0.772205,0.227795,3,68,10,180
4,0.000075,"(24734179100, 7003513941, 7102187693)",group5,"24734179100, 7003513941, 7102187693","[0011957585, 50449094016, 70349170306]","[85021017087, 0030764844, 85091217062, 0032509...","[countries differential access, disappearance ...","[location parasite extraparenchymal, marginali...",0.774753,0.225247,3,61,10,184
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17672,0.000075,"(50361055400, 57437439400, 57438019100, 572229...",group17673,"50361055400, 57437439400, 57438019100, 5722298...","[85125384992, 85127342869, 85131248040]","[85120535327, 85104328183, 85131537395, 851269...","[mitochondrial transfer transplant, work preve...","[hcps exist related, identification diversity ...",0.742257,0.257743,3,7,15,28
17673,0.000075,"(50361055400, 45961087900, 57437439400, 574380...",group17674,"50361055400, 45961087900, 57437439400, 5743801...","[85125384992, 85127342869, 85131248040]","[85082691209, 85131537395, 85091110208, 850616...","[mitochondrial transfer transplant, work preve...","[patch treat gastroschisis, highlight need sta...",0.721477,0.278523,3,30,15,149
17674,0.000075,"(50361055400, 45961087900, 57437439400, 572229...",group17675,"50361055400, 45961087900, 57437439400, 5722298...","[85125384992, 85127342869, 85131248040]","[85082691209, 85131537395, 85091110208, 850616...","[mitochondrial transfer transplant, work preve...","[patch treat gastroschisis, highlight need sta...",0.723147,0.276853,3,31,15,154
17675,0.000075,"(50361055400, 45961087900, 57438019100, 572229...",group17676,"50361055400, 45961087900, 57438019100, 5722298...","[85125384992, 85127342869, 85131248040]","[85082691209, 85131537395, 85091110208, 850616...","[mitochondrial transfer transplant, work preve...","[patch treat gastroschisis, highlight need sta...",0.723147,0.276853,3,31,15,154


In [44]:
# no tiene keywords ni en keywords_group ni en keywords_all
valid_groups_cleaned = valid_groups.dropna(subset=['keywords_group', 'keywords_all'], how='all')

valid_groups_cleaned = valid_groups_cleaned[
    (valid_groups_cleaned['keywords_group'].apply(lambda x: len(x) > 0)) |
    (valid_groups_cleaned['keywords_all'].apply(lambda x: len(x) > 0))
]

In [45]:
valid_groups_cleaned = valid_groups_cleaned[columns_to_save]

In [46]:
valid_groups_cleaned

,id_group,authors,id_articles_group,num_articles_group,id_articles_all,num_articles_all,keywords_group,num_keywords_group,keywords_all,num_keywords_all,similarity_cosine,index_novelty
0,group1,"7403483234, 57200218104, 6603155100","[84885079328, 84946204179, 84962109938]",3,"[84864296662, 85093931360, 85055531776, 851317...",170,"[ovt instructing new, acoustic flow monitors, ...",15,"[teleseismic travel time, reduced computationa...",744,0.835043,0.164957
1,group2,"24734179100, 7003513941, 7005237183","[0011957585, 50449094016, 70349170306]",3,"[85021017087, 0026699611, 0030764844, 85091217...",68,"[countries differential access, disappearance ...",10,"[location parasite extraparenchymal, marginali...",187,0.771965,0.228035
2,group3,"24734179100, 7005237183, 7102187693","[0011957585, 50449094016, 70349170306]",3,"[0027731713, 0026653512, 0026699611, 003180785...",13,"[countries differential access, disappearance ...",10,"[review article suggest, resistant epilepsy su...",10,0.582562,0.417438
3,group4,"7003513941, 7005237183, 7102187693","[0011957585, 50449094016, 70349170306]",3,"[85021017087, 0026699611, 0030764844, 85091217...",68,"[countries differential access, disappearance ...",10,"[location parasite extraparenchymal, marginali...",180,0.772205,0.227795
4,group5,"24734179100, 7003513941, 7102187693","[0011957585, 50449094016, 70349170306]",3,"[85021017087, 0030764844, 85091217062, 0032509...",61,"[countries differential access, disappearance ...",10,"[location parasite extraparenchymal, marginali...",184,0.774753,0.225247
...,...,...,...,...,...,...,...,...,...,...,...,...
17672,group17673,"50361055400, 57437439400, 57438019100, 5722298...","[85125384992, 85127342869, 85131248040]",3,"[85120535327, 85104328183, 85131537395, 851269...",7,"[mitochondrial transfer transplant, work preve...",15,"[hcps exist related, identification diversity ...",28,0.742257,0.257743
17673,group17674,"50361055400, 45961087900, 57437439400, 5743801...","[85125384992, 85127342869, 85131248040]",3,"[85082691209, 85131537395, 85091110208, 850616...",30,"[mitochondrial transfer transplant, work preve...",15,"[patch treat gastroschisis, highlight need sta...",149,0.721477,0.278523
17674,group17675,"50361055400, 45961087900, 57437439400, 5722298...","[85125384992, 85127342869, 85131248040]",3,"[85082691209, 85131537395, 85091110208, 850616...",31,"[mitochondrial transfer transplant, work preve...",15,"[patch treat gastroschisis, highlight need sta...",154,0.723147,0.276853
17675,group17676,"50361055400, 45961087900, 57438019100, 5722298...","[85125384992, 85127342869, 85131248040]",3,"[85082691209, 85131537395, 85091110208, 850616...",31,"[mitochondrial transfer transplant, work preve...",15,"[patch treat gastroschisis, highlight need sta...",154,0.723147,0.276853


In [47]:
valid_groups_cleaned.to_csv(output_path, index=False)
print(f"Archivo de grupos guardado en: {output_path}")

Archivo de grupos guardado en: C:/Users/Krlozz/Documents/Tesis/TesisFinal/ProcessedData/generated_groups.csv


# Gráficos info

In [48]:
from bokeh.plotting import figure, output_notebook, show
from bokeh.models import ColumnDataSource, HoverTool

In [49]:
output_notebook()

Loading BokehJS ...

In [50]:
file_path = 'C:/Users/Krlozz/Documents/Tesis/TesisFinal/ProcessedData/generated_groups.csv'
data = pd.read_csv(file_path)

In [61]:
data = data.dropna(subset=['num_keywords_group', 'num_keywords_all'], how='any')
data['num_keywords_group'] = data['num_keywords_group'].replace(0, pd.NA).dropna()
data['num_keywords_all'] = data['num_keywords_all'].replace(0, pd.NA).dropna()

In [62]:
data['num_authors'] = data['authors'].apply(lambda x: len(x.split(', ')) if pd.notna(x) else 0)
authors_min = data['num_authors'].min()
authors_max = data['num_authors'].max()

In [63]:
columns_to_plot = [
    'num_articles_group',
    'num_articles_all',
    'num_keywords_group',
    'num_keywords_all',
    'index_novelty',
    'num_authors'
]

In [64]:
for column in columns_to_plot:
    col_min = data[column].min()
    col_max = data[column].max()
    col_mean = data[column].mean()
    
    source = ColumnDataSource(data={
        'metric': ['min', 'mean', 'max'],
        'value': [col_min, col_mean, col_max]
    })
    
    p = figure(
        title=f"Métricas para {column}",
        x_range=['min', 'mean', 'max'],
        y_axis_label=column,
        plot_height=400,
        plot_width=600,
        toolbar_location="above"
    )
    
    p.vbar(x='metric', top='value', source=source, width=0.6, color='navy', alpha=0.7, legend_label=column)
    
    hover = HoverTool(tooltips=[("Métrica", "@metric"), ("Valor", "@value")])
    p.add_tools(hover)
    
    p.legend.visible = False
    show(p)

# Data group

In [86]:
import ast

In [87]:
input_path = "C:/Users/Krlozz/Documents/Tesis/TesisFinal/ProcessedData/generated_groups.csv"
output_path = "C:/Users/Krlozz/Documents/Tesis/TesisFinal/ProcessedData/data_groups.csv"

In [88]:
data = pd.read_csv(input_path)

In [89]:
def merge_keywords(row):
    
    try:
        keywords_group = ast.literal_eval(row['keywords_group']) if pd.notna(row['keywords_group']) else []
        keywords_all = ast.literal_eval(row['keywords_all']) if pd.notna(row['keywords_all']) else []
    except (ValueError, SyntaxError):
        keywords_group = []
        keywords_all = []
    
    all_keywords = set(keywords_group + keywords_all)
    return ','.join(all_keywords)

In [90]:
data['keywords'] = data.apply(merge_keywords, axis=1)

In [91]:
data_groups = data[['id_group', 'authors', 'keywords', 'index_novelty']]

In [92]:
data_groups.to_csv(output_path, index=False)
print(f"Archivo generado en: {output_path}")

Archivo generado en: C:/Users/Krlozz/Documents/Tesis/TesisFinal/ProcessedData/data_groups.csv
